In [1]:
import duckdb
import time

In [2]:
%%time
# Connect to DuckDB using persistence storage
con = duckdb.connect(database = "my-db.duckdb", read_only = False)

CPU times: total: 0 ns
Wall time: 22.6 ms


In [3]:
%%time
# Read the bitcoin dataset
bc_data = duckdb.read_csv('data/half1_BTCUSDT_1s.csv')

CPU times: total: 312 ms
Wall time: 433 ms


In [4]:
%%time
# Show the output of the bitcoin dataset
start_time = time.time()
duckdb.sql('''SELECT * FROM bc_data
''').show()
end_time = time.time()
print(f'Total time taken: {end_time - start_time}')

┌─────────────────────┬─────────┬─────────┬─────────┬───┬──────────────────────┬──────────────────────┬────────┐
│      Open Time      │  Open   │  High   │   Low   │ … │ Taker Buy Base Ass…  │ Taker Buy Quote As…  │ Ignore │
│      timestamp      │ double  │ double  │ double  │   │        double        │        double        │ int64  │
├─────────────────────┼─────────┼─────────┼─────────┼───┼──────────────────────┼──────────────────────┼────────┤
│ 2017-08-17 04:00:28 │ 4261.48 │ 4261.48 │ 4261.48 │ … │                  0.0 │                  0.0 │      0 │
│ 2017-08-17 04:00:29 │ 4261.48 │ 4261.48 │ 4261.48 │ … │                  0.0 │                  0.0 │      0 │
│ 2017-08-17 04:00:30 │ 4261.48 │ 4261.48 │ 4261.48 │ … │                  0.0 │                  0.0 │      0 │
│ 2017-08-17 04:00:31 │ 4261.48 │ 4261.48 │ 4261.48 │ … │                  0.0 │                  0.0 │      0 │
│ 2017-08-17 04:00:32 │ 4261.48 │ 4261.48 │ 4261.48 │ … │             0.075183 │         320.390

In [5]:
%%time
# Check the start time, end time and the total rows in the dataset
start_time = time.time()
duckdb.sql('''SELECT MIN("Open Time") as starting_time, 
                        MAX("Open Time") as ending_time,
                        COUNT(*)
                FROM bc_data
''').show()
end_time = time.time()
print(f'Total time taken: {end_time - start_time}')

┌─────────────────────┬─────────────────────┬──────────────┐
│    starting_time    │     ending_time     │ count_star() │
│      timestamp      │      timestamp      │    int64     │
├─────────────────────┼─────────────────────┼──────────────┤
│ 2017-08-17 04:00:28 │ 2021-02-23 09:20:19 │    110671732 │
└─────────────────────┴─────────────────────┴──────────────┘

Total time taken: 19.445404291152954
CPU times: total: 1min 18s
Wall time: 19.4 s


In [27]:
%%time
# Calculate the 50 seconds moving average and 200 seconds moving average and show the output
start_time = time.time()
duckdb.sql('''
                SELECT "Open Time",
                        Close,
                        AVG("Close") OVER ( ORDER BY "Open Time"  
                                            RANGE BETWEEN INTERVAL 50 SECONDS PRECEDING
                                              AND INTERVAL 0 SECONDS FOLLOWING) as ma50,
                        AVG("Close") OVER ( ORDER BY "Open Time" 
                                            RANGE BETWEEN INTERVAL 200 SECONDS PRECEDING
                                              AND INTERVAL 0 SECONDS FOLLOWING) as ma200
                FROM bc_data
''').show()
end_time = time.time()
print(f'Total time taken: {end_time - start_time}')

┌─────────────────────┬─────────┬───────────────────┬───────────────────┐
│      Open Time      │  Close  │       ma50        │       ma200       │
│      timestamp      │ double  │      double       │      double       │
├─────────────────────┼─────────┼───────────────────┼───────────────────┤
│ 2017-08-17 04:00:28 │ 4261.48 │           4261.48 │           4261.48 │
│ 2017-08-17 04:00:29 │ 4261.48 │           4261.48 │           4261.48 │
│ 2017-08-17 04:00:30 │ 4261.48 │           4261.48 │           4261.48 │
│ 2017-08-17 04:00:31 │ 4261.48 │           4261.48 │           4261.48 │
│ 2017-08-17 04:00:32 │ 4261.48 │           4261.48 │           4261.48 │
│ 2017-08-17 04:00:33 │ 4261.48 │           4261.48 │           4261.48 │
│ 2017-08-17 04:00:34 │ 4261.48 │           4261.48 │           4261.48 │
│ 2017-08-17 04:00:35 │ 4261.48 │           4261.48 │           4261.48 │
│ 2017-08-17 04:00:36 │ 4261.48 │           4261.48 │           4261.48 │
│ 2017-08-17 04:00:37 │ 4261.48 │ 4261

In [26]:
%%time
# Calculate the 50 seconds moving average and 200 seconds moving average and show the output
# Notice that query is not yet evaluated. It will be evaluated only when we call a method that triggers evaluation
start_time = time.time()
moving_averages = duckdb.sql('''
                SELECT "Open Time",
                        Close,
                        AVG("Close") OVER ( ORDER BY "Open Time"  
                                            RANGE BETWEEN INTERVAL 50 SECONDS PRECEDING
                                              AND INTERVAL 0 SECONDS FOLLOWING) as ma50,
                        AVG("Close") OVER ( ORDER BY "Open Time" 
                                            RANGE BETWEEN INTERVAL 200 SECONDS PRECEDING
                                              AND INTERVAL 0 SECONDS FOLLOWING) as ma200
                FROM bc_data
''')
end_time = time.time()
print(f'Total time taken: {end_time - start_time}')

Total time taken: 0.0
CPU times: total: 0 ns
Wall time: 0 ns


In [28]:
%%time
# Calculatin the maximum open time and minimum open time and the count of the records from the above query.
# The above query is evaluated and then below query is processed
start_time = time.time()
duckdb.sql('''
                SELECT MAX("Open Time"), MIN("Open Time"), count(*)
                FROM moving_averages

''').show()
end_time = time.time()
print(f'Total time taken: {end_time - start_time}')

┌─────────────────────┬─────────────────────┬──────────────┐
│  max("Open Time")   │  min("Open Time")   │ count_star() │
│      timestamp      │      timestamp      │    int64     │
├─────────────────────┼─────────────────────┼──────────────┤
│ 2021-02-23 09:20:19 │ 2017-08-17 04:00:28 │    110671732 │
└─────────────────────┴─────────────────────┴──────────────┘

Total time taken: 97.46205115318298
CPU times: total: 7min 38s
Wall time: 1min 37s


In [30]:
%%time
# Calculatin the maximum open time and minimum open time and the count of the records from the above query.
# The records are filter to only before '2017-08-18'
# The above query is evaluated and then below query is processed
start_time = time.time()
duckdb.sql('''
                SELECT MAX("Open Time"), MIN("Open Time"), count(*)
                FROM moving_averages
                WHERE "Open Time" < '2017-08-18 00:00:00'

''').show()
end_time = time.time()
print(f'Total time taken: {end_time - start_time}')

┌─────────────────────┬─────────────────────┬──────────────┐
│  max("Open Time")   │  min("Open Time")   │ count_star() │
│      timestamp      │      timestamp      │    int64     │
├─────────────────────┼─────────────────────┼──────────────┤
│ 2017-08-17 23:59:59 │ 2017-08-17 04:00:28 │        71972 │
└─────────────────────┴─────────────────────┴──────────────┘

Total time taken: 99.59736180305481
CPU times: total: 7min 50s
Wall time: 1min 39s


In [20]:
%%time
# Checking the data in the moving averages table
start_time = time.time()
duckdb.sql('''
                SELECT *
                FROM moving_averages
''').show()
end_time = time.time()
print(f'Total time taken: {end_time - start_time}')

┌─────────────────────┬─────────┬───────────────────┬───────────────────┐
│      Open Time      │  Close  │       ma50        │       ma200       │
│      timestamp      │ double  │      double       │      double       │
├─────────────────────┼─────────┼───────────────────┼───────────────────┤
│ 2017-08-17 04:00:28 │ 4261.48 │           4261.48 │           4261.48 │
│ 2017-08-17 04:00:29 │ 4261.48 │           4261.48 │           4261.48 │
│ 2017-08-17 04:00:30 │ 4261.48 │           4261.48 │           4261.48 │
│ 2017-08-17 04:00:31 │ 4261.48 │           4261.48 │           4261.48 │
│ 2017-08-17 04:00:32 │ 4261.48 │           4261.48 │           4261.48 │
│ 2017-08-17 04:00:33 │ 4261.48 │           4261.48 │           4261.48 │
│ 2017-08-17 04:00:34 │ 4261.48 │           4261.48 │           4261.48 │
│ 2017-08-17 04:00:35 │ 4261.48 │           4261.48 │           4261.48 │
│ 2017-08-17 04:00:36 │ 4261.48 │           4261.48 │           4261.48 │
│ 2017-08-17 04:00:37 │ 4261.48 │ 4261